In [18]:
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler

Dataset basic operations

In [19]:
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [20]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [21]:
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In [22]:
dataset = dataset.map(lambda x: x * 2)
for item in dataset:
    print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int32)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
tf.Tensor([16 18], shape=(2,), dtype=int32)


In [23]:
dataset = dataset.filter(lambda x: tf.reduce_sum(x) > 50)
for item in dataset:
    print(item)

tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)


In [24]:
for item in dataset.take(2):
    print(item)

tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)


Shuffle

In [25]:
dataset = dataset.repeat(3)
dataset = dataset.shuffle(buffer_size=4, seed=42)
for item in dataset:
    print(item)

tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)


In [26]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

In [27]:
scaler = StandardScaler()
scaler.fit(X_train)
X_mean, X_std = scaler.mean_, scaler.scale_

In [28]:
def save_to_csv(data, dataset_name, prefix, header, chunks_num):
    main_dir = Path() / 'datasets' / dataset_name
    main_dir.mkdir(parents=True, exist_ok=True)
    filename = "{}_{}_{:02d}.csv" # dataset name, prefix, file_num

    all_files = []
    for idx, chunk in enumerate(np.array_split(data, chunks_num)):
        file_path = main_dir / filename.format(dataset_name, prefix, idx)
        with open(file_path, 'w') as file:
            if header is not None:
                file.write(header + '\n')
            for row in chunk:
                file.write(','.join([repr(col) for col in row]) + '\n')
        all_files.append(str(file_path))
    
    return all_files

In [29]:
train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]
header_cols = housing.feature_names + ["MedianHouseValue"]
header = ",".join(header_cols)
dataset_name = 'housing'

train_filepaths = save_to_csv(train_data, dataset_name, "train", header, 20)
valid_filepaths = save_to_csv(valid_data, dataset_name, "valid", header, 10)
test_filepaths = save_to_csv(test_data, dataset_name, "test", header, 10)

Reading from multiple files

In [30]:
# dataset with file paths
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)

In [31]:
n_readers = 5
dataset = filepath_dataset.interleave(lambda filepath: tf.data.TextLineDataset(filepath).skip(1), cycle_length=n_readers)

In [32]:
for line in dataset.take(5):
    print(line)

tf.Tensor(b'4.2083,44.0,5.323204419889502,0.9171270718232044,846.0,2.3370165745856353,37.47,-122.2,2.782', shape=(), dtype=string)
tf.Tensor(b'4.1812,52.0,5.701388888888889,0.9965277777777778,692.0,2.4027777777777777,33.73,-118.31,3.215', shape=(), dtype=string)
tf.Tensor(b'3.6875,44.0,4.524475524475524,0.993006993006993,457.0,3.195804195804196,34.04,-118.15,1.625', shape=(), dtype=string)
tf.Tensor(b'3.3456,37.0,4.514084507042254,0.9084507042253521,458.0,3.2253521126760565,36.67,-121.7,2.526', shape=(), dtype=string)
tf.Tensor(b'3.5214,15.0,3.0499445061043287,1.106548279689234,1447.0,1.6059933407325193,37.63,-122.43,1.442', shape=(), dtype=string)


Data preprocessing

In [33]:
n_inputs = 8

def parse_csv_line(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    return tf.stack(fields[:-1]), tf.stack(fields[-1:])

def preprocess(line):
    x, y = parse_csv_line(line)
    return (x - X_mean) / X_std, y

In [34]:
processed_dataset = dataset.map(lambda line: preprocess(line))
for line in processed_dataset.take(5):
    print(line)

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
array([ 0.36618188, -0.998705  ,  0.00781878, -0.00675364, -0.06140145,
        0.0072037 , -0.94465536,  0.9367464 ], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.418], dtype=float32)>)
(<tf.Tensor: shape=(8,), dtype=float32, numpy=
array([-0.7398411 , -0.36583957, -0.78467995,  0.07414506,  0.75447065,
        0.40770054, -0.68699217,  0.60190356], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.], dtype=float32)>)
(<tf.Tensor: shape=(8,), dtype=float32, numpy=
array([ 0.19852655,  1.2954322 , -0.13117608, -0.31340504, -0.8508079 ,
       -0.04867317,  0.85898864, -1.3022108 ], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.67], dtype=float32)>)
(<tf.Tensor: shape=(8,), dtype=float32, numpy=
array([-0.8936168 ,  0.9789995 , -0.6810549 , -0.07264194, -0.5669866 ,
       -0.39212456, -1.3522334 ,  1.2316071 ], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, nu